- 回归估计一个连续值
    - 单连续值输出，跟真实值的区别作为损失
- 分类预测一个离散类别
    - 通常是多输出，输出值i是预测为i类的置信度

本质上，你也可以使用均方误差作为损失函数，但是问题在于，我们在进行分类的时候，我们并不关系，非正确量的预测值有多大
我们关心的是正确量的预测值要足够的大
如果依然使用均方误差，那么就走错路了，因为，其他非关键因素的误差会影响我梯度下降的方向
在线性回归中，我们认为一个线性方程的输出值是一个，所以，一般输出维数就是1

但是在多分类中，如果还是使用线性层进行计算，那么输出维数就和你想要表达的分类数是相关的，这个的表达就是2 进 2 出的模型
$$
O_1 = \omega_{11}x+ \omega_{12}x+b_1

$$

$$
O_2 = \omega_{21}x+ \omega_{22}x+b_2
$$

选择这样的代码是因为，指数能够使得任何的输入都输出一个正数

# SoftMax的运算

我们希望我的输出值y_hat可以直接表示类i的概率，那么需要对输出值进行规范化（即限制输出值之和为1）

 softmax函数将未规范化的预测变换为非负并且总和为1，同时要求模型保持可导。

$$
  \hat y = softmax(o) 这个的意思是，对线性层输出值进行softmax计算，将该值作为我模型的输出值，其中详细的写法如下
$$



$$
  \hat y_i = \frac {exp(o_i)}{\sum_{k}exp(o_k)}
$$



尽管softmax函数是一个非线性函数，但是softmaxt回归的输出仍然由输入特征的仿射变换决定，因此softmax回归是一个线性模型 Linear model


## 开始实践
MNIST数据集是图像分类中广泛使用的数据集，但是作为基准数据集过于简单，我们将使用更加复杂的Fashion-MNIST数据集

In [2]:
import torch
import torchvision
from torch.utils import data #数据读取
from torchvision import transforms #数据操作
from d2l import torch as d2l

d2l.use_svg_display()

In [3]:
trans = transforms.ToTensor() # 将数据集图片转成Tensor类型，将图像数据从PIL类型转变成32位浮点数格式
# 并除以255 使得图像的数值都保持在0-1之间
mnist_train = torchvision.datasets.FashionMNIST(
    root="./data",
    train=True,
    transform=trans,
    download=True)
#transform=trans, 转成tensor
# 测试数据集，不参与训练，但是验证数据的好坏
mnist_test = torchvision.datasets.FashionMNIST(root="./data", train=False, transform=trans, download=True)
len(mnist_train),len(mnist_test)

(60000, 10000)

In [4]:
mnist_train[0][0].shape  # 第一张图片，灰白图片，channel =1，其他的就是像素值

torch.Size([1, 28, 28])

In [5]:
# 数字索引标签到真实标签的对应关系
def get_fashion_mnist_labels(labels):  #@save
    """返回Fashion-MNIST数据集的文本标签"""
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

In [6]:
# 图像绘制
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):  #@save
    """绘制图像列表"""
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if torch.is_tensor(img):
            # 图片张量
            ax.imshow(img.numpy())
        else:
            # PIL图片
            ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

In [7]:
# X,y = next(iter(data.DataLoader(mnist_train,batch_size=18)))
# show_images(X.reshape(18,28,28),2,9,titles=get_fashion_mnist_labels(y))

In [11]:
batch_size = 256
def get_dataloader_workers():
    """因为数据是放在硬盘上的，数据的读取并不方便，所以需要多线程进行数据的读取"""
    return 4


train_iter = data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=get_dataloader_workers())
timer= d2l.Timer()
for X,y in train_iter:
    continue
f'{timer.stop():.2f} sec'
# benchMark数据读取

'4.72 sec'

In [8]:
def load_data_fashion_mnist(batchsize, resize = None):
    #ToTensor()将shape为(H, W, C)的nump.ndarray或img转为shape为(C, H, W)的tensor，其将每一个数值归一化到[0,1]，
    # 其归一化方法比较简单，直接除以255即可。具体可参见如下代码：
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0,transforms.Resize(resize))
    trans = transforms.Compose(trans) # 将多个操作连续执行，可以传入多个操作
    minist_train = torchvision.datasets.FashionMNIST(root="./data", train=True, transform=trans, download=False)
    minist_test = torchvision.datasets.FashionMNIST(root="./data", train=False, transform=trans, download=False)
    return (data.DataLoader(minist_train,batch_size,True,num_workers=get_dataloader_workers()),
            data.DataLoader(minist_test,batch_size,True,num_workers=get_dataloader_workers())
            )

SoftMax的从0实现